<a href="https://colab.research.google.com/github/sunyux/VGG16-and-Retnet50-for-MNIST/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torchvision
from torchvision import datasets, transforms 
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from copy import deepcopy

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

batch_size = 100


trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)



In [ ]:
pretrained_net = torchvision.models.resnet50(pretrained=True)

In [ ]:
#for the vgg 16
pretrained_net2 = torchvision.models.vgg16(pretrained=True)

In [ ]:
pretrained_net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = pretrained_net.fc.in_features
pretrained_net.fc = nn.Linear(num_ftrs, 10)
layers = 0
for child in pretrained_net.children():
        layers += 1
        if layers < 9:
            for param in child.parameters():
                param.requires_grad = False


In [ ]:
#def change_layers(pretrained_net):
pretrained_net2.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
num_ftrs = pretrained_net2.classifier[6].in_features
pretrained_net2.classifier[6] = nn.Linear(num_ftrs, 10)

layers = 0
for child in pretrained_net2.classifier:
        layers += 1
        if layers < 6:
            for param in child.parameters():
                param.requires_grad = False


In [ ]:
print(pretrained_net2)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
print(pretrained_net)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets


    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
  
        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
def get_custom_data(size, batch_size):
    img_index = 0
    label_index = 1

    counter = np.zeros(10)
    customDataset = CustomDataset([], [])

    trainset, testset = get_data(batch_size)

    for data in trainset:
        image = data[img_index]
        label = data[label_index]

        if counter[label] <= size:
            customDataset.data.append(deepcopy(image))
            customDataset.targets.append(deepcopy(label))
            counter[label] += 1

        if sum(counter) >= size*10:
            break

    trainloader = torch.utils.data.DataLoader(customDataset, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

    return trainloader, testloader

In [ ]:
def get_data(batch_size):
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5)
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5)
        ]),
    }

    trainset = datasets.MNIST('./data', download=True, train=True, transform=data_transforms['train'])
    testset = datasets.MNIST('./data', download=True, train=False, transform=data_transforms['val'])

    return trainset, testset

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_net.parameters(), lr=0.001, momentum=0)

In [ ]:
from torch.cuda import is_available
                                                                                                                                                                                                                                                                                                 #train the model
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    model.train()
    model.eval()
    for batch, data in enumerate(dataloader):
        images, labels=data
        if(torch.cuda.is_available()):
          images=images
          labels=labels

        # Compute prediction error
        pred = model(images)
        _,pred_label = torch.max(pred.data, dim = 1)
        loss = criterion(pred, labels)
      
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(images)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
#test the model
test_error=[]
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
          #  X, y = X.to(device), y.to(device)
            pred = model(X)
           # test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    #test_loss /= num_batches
    correct /= size
    test_error.append(1-correct)
    print(f"Accuracy: {(100*correct):>0.1f}%")



In [ ]:
trainloader_class, testloader_class = get_custom_data(50, batch_size)
epochs = 80
for t in range(epochs):
    print(f"Epoch  {t+1}\n-------------------------------")
    train(trainloader_class, pretrained_net, criterion, optimizer)
    test(testloader_class, pretrained_net, criterion)
print("Done!")

Epoch  1
-------------------------------
loss: 2.290380  [    0/  500]
Accuracy: 14.0%
Epoch  2
-------------------------------
loss: 2.321535  [    0/  500]
Accuracy: 14.1%
Epoch  3
-------------------------------
loss: 2.270359  [    0/  500]
Accuracy: 14.0%
Epoch  4
-------------------------------
loss: 2.298172  [    0/  500]
Accuracy: 14.1%
Epoch  5
-------------------------------
loss: 2.296289  [    0/  500]
Accuracy: 14.2%
Epoch  6
-------------------------------
loss: 2.286207  [    0/  500]
Accuracy: 14.2%
Epoch  7
-------------------------------
loss: 2.266044  [    0/  500]
Accuracy: 14.4%
Epoch  8
-------------------------------
loss: 2.271939  [    0/  500]
Accuracy: 14.8%
Epoch  9
-------------------------------
loss: 2.273991  [    0/  500]
Accuracy: 15.5%
Epoch  10
-------------------------------
loss: 2.250826  [    0/  500]
